# Noisy QCNN Fashion MNIST

In [1]:
from pennylane import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import sys
sys.path.insert(0, '/Users/tak/Github/QEmbedding/')
import Hybrid_nn
import torch
from torch import nn
import data
import pennylane as qml
import embedding

IBMQ Fake provider

In [2]:
import pennylane as qml
from pennylane import numpy as np
from qiskit import IBMQ
from qiskit.providers.fake_provider import FakeJakarta, FakeMontreal
from qiskit_aer.noise import NoiseModel


noisy = FakeJakarta()
noise_model = NoiseModel.from_backend(noisy)
coupling_map = noisy.configuration().coupling_map
basis_gates = noise_model.basis_gates

dev_fake = qml.device(
    'qiskit.aer',
    wires=4,
    shots=1024,
    noise_model=noise_model,
    coupling_map=coupling_map,
    basis_gates=basis_gates
)

# NQE Optimization

In [3]:
@qml.qnode(dev_fake, interface="torch")
def Noisy_Four_circuit2(inputs): 
    embedding.Noisy_Four_QuantumEmbedding2(inputs[0:7])
    embedding.Noisy_Four_QuantumEmbedding2_inverse(inputs[7:14])
    return qml.probs(wires=range(4))

class PCA_NQE(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.qlayer2 = qml.qnn.TorchLayer(Noisy_Four_circuit2, weight_shapes={})
        self.linear_relu_stack2 = nn.Sequential(
            nn.Linear(4, 12),
            nn.ReLU(),
            nn.Linear(12, 12),
            nn.ReLU(),
            nn.Linear(12, 7)
        )
    def forward(self, x1, x2):
        x1 = self.linear_relu_stack2(x1)
        x2 = self.linear_relu_stack2(x2)
        x = torch.concat([x1, x2], 1)
        x = self.qlayer2(x)
        return x[:,0]


class NQE(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.qlayer3 = qml.qnn.TorchLayer(Noisy_Four_circuit2, weight_shapes={})
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Layer2: 14 * 14 -> 7 * 7
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Fully connected Layers 7 * 7 -> 7
        self.fc = torch.nn.Linear(7 * 7, 7, bias=True)
        
    def forward(self, x1, x2):
        x1 = self.layer1(x1)
        x1 = self.layer2(x1)
        x1 = x1.view(-1, 7 * 7)
        x1 = self.fc(x1)

        x2 = self.layer1(x2)
        x2 = self.layer2(x2)
        x2 = x2.view(-1, 7 * 7)
        x2 = self.fc(x2)

        x = torch.concat([x1, x2], 1)
        x = self.qlayer3(x)
        return x[:,0]

In [4]:
def new_data(batch_size, X, Y, feature_reduction):
    X1_new, X2_new, Y_new = [], [], []
    for i in range(batch_size):
        n, m = np.random.randint(len(X)), np.random.randint(len(X))
        X1_new.append(X[n])
        X2_new.append(X[m])
        if Y[n] == Y[m]:
            Y_new.append(1)
        else:
            Y_new.append(0)

    X1_new, X2_new, Y_new = torch.tensor(X1_new).to(torch.float32), torch.tensor(X2_new).to(torch.float32), torch.tensor(Y_new).to(torch.float32)
    if feature_reduction == False:
        X1_new = X1_new.permute(0, 3, 1, 2)
        X2_new = X2_new.permute(0, 3, 1, 2)
    return X1_new, X2_new, Y_new

def train_models(model_name, X_train, Y_train, feature_reduction):
    if model_name == 'PCA-NQE':
        model = PCA_NQE()
    elif model_name == 'NQE':
        model = NQE()

    model.train()

    loss_fn = torch.nn.MSELoss()
    opt = torch.optim.SGD(model.parameters(), lr=0.01)
    for it in range(1):

        X1_batch, X2_batch, Y_batch = new_data(25, X_train, Y_train, feature_reduction)
        X1_batch, X2_batch, Y_batch = X1_batch, X2_batch, Y_batch

        pred = model(X1_batch, X2_batch)
        pred, Y_batch = pred.to(torch.float32), Y_batch.to(torch.float32)
        loss = loss_fn(pred, Y_batch)

        opt.zero_grad()
        loss.backward()
        opt.step()

        if it % 50 == 0:
            print(f"Iterations: {it} Loss: {loss.item()}")
            PATH = f'{model_name}{it}.pt'
            torch.save(model.state_dict(), PATH)





In [5]:
feature_reduction = 'PCA4'
classes = [0,1]
X_train, X_test, Y_train, Y_test = data.data_load_and_process('fashion', feature_reduction=feature_reduction, classes=classes)
train_models('PCA-NQE', X_train, Y_train, feature_reduction)

/var/folders/s2/t3n82l2s329dh9dttmtv7vr00000gn/T/ipykernel_2717/89923749.py:12: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:248.)
  X1_new, X2_new, Y_new = torch.tensor(X1_new).to(torch.float32), torch.tensor(X2_new).to(torch.float32), torch.tensor(Y_new).to(torch.float32)


Iterations: 0 Loss: 0.2722699046134949


In [6]:
feature_reduction = False
classes = [0,1]
X_train, X_test, Y_train, Y_test = data.data_load_and_process('fashion', feature_reduction=feature_reduction, classes=classes)
train_models('NQE', X_train, Y_train, feature_reduction)

Iterations: 0 Loss: 0.2845287322998047


# Trace Distance History

In [ ]:
dev = qml.device('default.qubit', wires=4)

X1_test, X0_test = [], []
for i in range(len(X_test)):
    if Y_test[i] == 1:
        X1_test.append(X_test[i])
    else:
        X0_test.append(X_test[i])
X1_test, X0_test = torch.tensor(X1_test).to(torch.float32), torch.tensor(X0_test).to(torch.float32)

X1_train, X0_train = [], []
for i in range(len(X_train)):
    if Y_train[i] == 1:
        X1_train.append(X_train[i])
    else:
        X0_train.append(X_train[i])
X1_train, X0_train = torch.tensor(X1_train).to(torch.float32), torch.tensor(X0_train).to(torch.float32)


@qml.qnode(dev, interface="torch")
def Noisy_Distance1(inputs): 
    embedding.Noisy_Four_QuantumEmbedding1(inputs)
    return qml.density_matrix(wires=range(4))

@qml.qnode(dev, interface="torch")
def Noisy_Distance2(inputs): 
    embedding.Noisy_Four_QuantumEmbedding2(inputs)
    return qml.density_matrix(wires=range(4))


class Distances1(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.qlayer1 = qml.qnn.TorchLayer(Noisy_Distance1, weight_shapes={})
        self.linear_relu_stack1 = nn.Sequential(
            nn.Linear(4, 6),
            nn.ReLU(),
            nn.Linear(6, 6),
            nn.ReLU(),
            nn.Linear(6, 4)
        )
    def forward(self, x1, x0):
        x1 = self.linear_relu_stack1(x1)
        x0 = self.linear_relu_stack1(x0)
        rhos1 = self.qlayer1(x1)
        rhos0 = self.qlayer1(x0)

        rho1 = torch.sum(rhos1, dim=0) / len(x1)
        rho0 = torch.sum(rhos0, dim=0) / len(x0)
        rho_diff = rho1 - rho0
        eigvals = torch.linalg.eigvals(rho_diff)
        return 0.5 * torch.real(torch.sum(torch.abs(eigvals)))


class Distances2(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.qlayer2 = qml.qnn.TorchLayer(Noisy_Distance2, weight_shapes={})
        self.linear_relu_stack2 = nn.Sequential(
            nn.Linear(4, 12),
            nn.ReLU(),
            nn.Linear(12, 12),
            nn.ReLU(),
            nn.Linear(12, 7)
        )
    def forward(self, x1, x0):
        x1 = self.linear_relu_stack2(x1)
        x0 = self.linear_relu_stack2(x0)
        rhos1 = self.qlayer2(x1)
        rhos0 = self.qlayer2(x0)

        rho1 = torch.sum(rhos1, dim=0) / len(x1)
        rho0 = torch.sum(rhos0, dim=0) / len(x0)
        rho_diff = rho1 - rho0
        eigvals = torch.linalg.eigvals(rho_diff)
        return 0.5 * torch.real(torch.sum(torch.abs(eigvals)))


class Distances3(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.qlayer3 = qml.qnn.TorchLayer(Noisy_Distance2, weight_shapes={})
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Layer2: 14 * 14 -> 7 * 7
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Fully connected Layers 7 * 7 -> 7
        self.fc = torch.nn.Linear(7 * 7, 7, bias=True)
    
    def forward(self, x1, x0):
        x1 = self.layer1(x1)
        x1 = self.layer2(x1)
        x1 = x1.view(-1, 7 * 7)
        x1 = self.fc(x1)

        x0 = self.layer1(x0)
        x0 = self.layer2(x0)
        x0 = x0.view(-1, 7 * 7)
        x0 = self.fc(x0)

        rhos1 = self.qlayer3(x1)
        rhos0 = self.qlayer3(x0)
        
        rho1 = torch.sum(rhos1, dim=0) / len(x1)
        rho0 = torch.sum(rhos0, dim=0) / len(x0)
        rho_diff = rho1 - rho0
        eigvals = torch.linalg.eigvals(rho_diff)
        return 0.5 * torch.real(torch.sum(torch.abs(eigvals)))


PATH_Model1 = '/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Noisy/Noisy_Model1_Fidelity.pt'
PATH_Model2 = '/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Noisy/Noisy_Model2_Fidelity.pt'
PATH_Model3 = '/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Noisy/Noisy_Model3_Fidelity.pt'
Model1_Fidelity_Distance = Distances1()
Model1_Fidelity_Distance.load_state_dict(torch.load(PATH_Model1, map_location=device))
Model2_Fidelity_Distance = Distances2()
Model2_Fidelity_Distance.load_state_dict(torch.load(PATH_Model2, map_location=device))

# Calculated from Model1 
Trace_before_traindata = 0.2734867334365845
Trace_before_testdata = 0.2723434865474701
LB_before_traindata = 0.5 * (1 - Trace_before_traindata)

print(f"Trace Distance (Training Data) Before: {Trace_before_traindata}")
print(f"Trace Distance (Test Data) Before: {Trace_before_testdata}")

# Distances After training with Model2_Fidelity
Trace_Model1_traindata = Model1_Fidelity_Distance(X1_train, X0_train)
Trace_Model1_testdata = Model1_Fidelity_Distance(X1_test, X0_test)
print(f"Trace Distance (Training Data) After Noisy Model1 Fidelity: {Trace_Model1_traindata}")
print(f"Trace Distance (Test Data) After Noisy Model1 Fidelity: {Trace_Model1_testdata}")

# Distances After training with Model2_HSinner
Trace_Model2_traindata = Model2_Fidelity_Distance(X1_train, X0_train)
Trace_Model2_testdata = Model2_Fidelity_Distance(X1_test, X0_test)
print(f"Trace Distance (Training Data) After Noisy Model2 Fidelity: {Trace_Model2_traindata}")
print(f"Trace Distance (Test Data) After Noisy Model2 Fidelity: {Trace_Model2_testdata}")

# Lower Bounds
LB_before_traindata = 0.5 * (1 - Trace_before_traindata)
LB_Model1_traindata = 0.5 * (1 - Trace_Model1_traindata.detach().numpy())
LB_Model2_traindata = 0.5 * (1 - Trace_Model2_traindata.detach().numpy())

In [ ]:
LB_before_traindata = 0.3633
LB_Model1_traindata = 0.0434
LB_Model2_traindata = 0.1066

Train Noisy QCNN classifier for Model1,2

In [ ]:
X_train, X_test = torch.tensor(X_train).to(torch.float32), torch.tensor(X_test).to(torch.float32)
Y_train = [-1 if y == 0 else 1 for y in Y_train]
Y_test = [-1 if y == 0 else 1 for y in Y_test]

In [ ]:
# Model1
class x_transform1(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack1 = nn.Sequential(
            nn.Linear(4, 6),
            nn.ReLU(),
            nn.Linear(6, 6),
            nn.ReLU(),
            nn.Linear(6, 4)
        )

    def forward(self, x):
        x = self.linear_relu_stack1(x)
        return x.detach().numpy()

# Model2
class x_transform2(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack2 = nn.Sequential(
            nn.Linear(4, 12),
            nn.ReLU(),
            nn.Linear(12, 12),
            nn.ReLU(),
            nn.Linear(12, 7)
        )
        
    def forward(self, x):
        x = self.linear_relu_stack2(x)
        return x.detach().numpy()

# Model3
class x_transform3(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Layer2: 14 * 14 -> 7 * 7
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Fully connected Layers 7 * 7 -> 7
        self.fc = torch.nn.Linear(7 * 7, 7, bias=True)
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(-1, 7 * 7)
        x = self.fc(x)
        return x.detach().numpy()

# Model Amplitude
class x_transform4(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, 32),
            nn.ReLU(),
            nn.Linear(32, 16)
        )
    def forward(self, x):
        x = self.linear_relu_stack(x)
        return x.detach().numpy()

model1 = x_transform1()
model2 = x_transform2()
model3 = x_transform3()
model_amp = x_transform4()

Tunable Hyperparameters

In [ ]:
steps = 1000
learning_rate = 0.01
batch_size = 25
ansatz = 'TTN'
#ansatz = 'SU4'

PATH_Model1 = '/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Noisy/Noisy_Model1_Fidelity.pt'
PATH_Model2 = '/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Noisy/Noisy_Model2_Fidelity.pt'
PATH_Model3 = '/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Noisy/Noisy_Model3_Fidelity.pt'
PATH_Model_amp = "/Users/tak/Github/QEmbedding/Results/earlystop 10 experiments/experiment1/Model Amplitude/Model_Amplitude.pt"

In [ ]:
def statepreparation(x, Trained):
    if Trained == False:
        embedding.Noisy_Four_QuantumEmbedding1(x)
    elif Trained == 'Model1_Fidelity':
        model1.load_state_dict(torch.load(PATH_Model1, map_location=device))
        x = model1(torch.tensor(x))
        embedding.Noisy_Four_QuantumEmbedding1(x)
    elif Trained == 'Model2_Fidelity':
        model2.load_state_dict(torch.load(PATH_Model2, map_location=device))
        x = model2(torch.tensor(x))
        embedding.Noisy_Four_QuantumEmbedding2(x)
    elif Trained == 'Model3_Fidelity':
        model3.load_state_dict(torch.load(PATH_Model3, map_location=device))
        x = model3(torch.tensor(x))
        embedding.Noisy_Four_QuantumEmbedding2(x[0])
    elif Trained == 'Model_amp' or Trained == 'Model_amp_not_trained':
        if Trained == 'Model_amp':
            model_amp.load_state_dict(torch.load(PATH_Model_amp, map_location=device))
            x = model_amp(torch.tensor(x))
        x = x / np.linalg.norm(x)
        qml.MottonenStatePreparation(x, wires=range(4))

    

@qml.qnode(dev_fake)
def Noisy_QCNN_classifier(params, x, Trained):
    statepreparation(x, Trained)
    embedding.Noisy_QCNN_four(params, ansatz)
    return qml.expval(qml.PauliZ(2))


def Linear_Loss(labels, predictions):
    loss = 0
    for l,p in zip(labels, predictions):
        loss += 0.5 * (1 - l * p)
    return loss / len(labels)


def cost(weights, X_batch, Y_batch, Trained):
    preds = [Noisy_QCNN_classifier(weights, x, Trained) for x in X_batch]
    return Linear_Loss(Y_batch, preds)


def circuit_training(X_train, Y_train, Trained):

    if ansatz == 'SU4':
        num_weights = 30
    elif ansatz == 'TTN':
        num_weights = 4

    weights = np.random.random(num_weights, requires_grad = True)
    opt = qml.NesterovMomentumOptimizer(stepsize=learning_rate)
    loss_history = []
    for it in range(steps):
        batch_index = np.random.randint(0, len(X_train), (batch_size,))
        X_batch = [X_train[i] for i in batch_index]
        Y_batch = [Y_train[i] for i in batch_index]
        weights, cost_new = opt.step_and_cost(lambda v: cost(v, X_batch, Y_batch, Trained),
                                                     weights)
        loss_history.append(cost_new)
        if it % 200 == 0:
            print("iteration: ", it, " cost: ", cost_new)
    return loss_history, weights

Train Model1&2 and save the results

In [ ]:
"""
Loss_histories_not_trained, weights_not_trained, Loss_histories_Model1_Fidelity, weights_Model1_Fidelity, Loss_histories_Model2_Fidelity, weights_Model2_Fidelity = [], [], [], [], [], []
for i in range(5):
    loss_not_trained, weight_not_trained = circuit_training(X_train, Y_train, False)
    loss_Model1_Fidelity, weight_Model1_Fidelity = circuit_training(X_train, Y_train, 'Model1_Fidelity')
    loss_Model2_Fidelity, weight_Model2_Fidelity = circuit_training(X_train, Y_train, 'Model2_Fidelity')

    Loss_histories_not_trained.append(loss_not_trained)
    Loss_histories_Model1_Fidelity.append(loss_Model1_Fidelity)
    Loss_histories_Model2_Fidelity.append(loss_Model2_Fidelity)

    weights_not_trained.append(weight_not_trained)
    weights_Model1_Fidelity.append(weight_Model1_Fidelity)
    weights_Model2_Fidelity.append(weight_Model2_Fidelity)

    
f = open('/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Noisy/Loss_histories_and_weights_Model1&2_Fidelity_TTN.txt', 'a')
for i in range(5):
    f.write(f'Loss History Noisy Model1 Fidelity {i + 1}:')
    f.write('\n')
    f.write(str(Loss_histories_Model1_Fidelity[i]))
    f.write('\n')
for i in range(5):
    f.write(f'Weights Noisy Model1 Fidelity {i + 1}:')
    f.write('\n')
    f.write(str(weights_Model1_Fidelity[i]))


for i in range(5):
    f.write(f'Loss History Noisy Model2 Fidelity {i + 1}:')
    f.write('\n')
    f.write(str(Loss_histories_Model2_Fidelity[i]))
    f.write('\n')
for i in range(5):
    f.write(f'Weights Noisy Model2 Fidelity {i + 1}:')
    f.write('\n')
    f.write(str(weights_Model2_Fidelity[i]))
f.close()
"""


# Part 2: Noisy Model3

Load the dataset for Noisy Model3

In [ ]:
feature_reduction = False
X_train, X_test, Y_train, Y_test = data.data_load_and_process('mnist', feature_reduction=feature_reduction, classes=classes)

Pre-training the embedding

In [ ]:
#train_models('Noisy_Model3_Fidelity')

Check the distances after pre-training Noisy Model3 Fidelity

In [ ]:
"""
X1_test, X0_test = [], []
for i in range(len(X_test)):
    if Y_test[i] == 1:
        X1_test.append(X_test[i])
    else:
        X0_test.append(X_test[i])
X1_test, X0_test = torch.tensor(X1_test).to(torch.float32), torch.tensor(X0_test).to(torch.float32)
X1_test, X0_test = X1_test.permute(0, 3, 1, 2), X0_test.permute(0, 3, 1, 2)

X1_train, X0_train = [], []
for i in range(len(X_train)):
    if Y_train[i] == 1:
        X1_train.append(X_train[i])
    else:
        X0_train.append(X_train[i])
X1_train, X0_train = torch.tensor(X1_train).to(torch.float32), torch.tensor(X0_train).to(torch.float32)
X1_train, X0_train = X1_train.permute(0, 3, 1, 2), X0_train.permute(0, 3, 1, 2)



PATH_Model3 = '/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Noisy/Noisy_Model3_Fidelity.pt'
Model3_Fidelity_Distance = Distances3()
Model3_Fidelity_Distance.load_state_dict(torch.load(PATH_Model3, map_location=device))


# Distances After training with Model2_Fidelity
Trace_Model3_traindata = Model3_Fidelity_Distance(X1_train, X0_train)
Trace_Model3_testdata = Model3_Fidelity_Distance(X1_test, X0_test)
print(f"Trace Distance (Training Data) After Noisy Model3 Fidelity: {Trace_Model3_traindata}")
print(f"Trace Distance (Test Data) After Noisy Model3 Fidelity: {Trace_Model3_testdata}")

# Lower Bounds
LB_Model3_traindata = 0.5 * (1 - Trace_Model3_traindata.detach().numpy())
"""

In [ ]:
LB_Model3_traindata = 0.0647

Noisy QCNN classifiers training for Model3

In [ ]:
X_train, X_test = torch.tensor(X_train).to(torch.float32), torch.tensor(X_test).to(torch.float32)
X_train, X_test = X_train.permute(0, 3, 1, 2), X_test.permute(0, 3, 1, 2)
Y_train = [-1 if y == 0 else 1 for y in Y_train]
Y_test = [-1 if y == 0 else 1 for y in Y_test]

Train Model3 and save the results

In [ ]:

"""
Loss_histories_Model3_Fidelity, weights_Model3_Fidelity = [], []
for i in range(5):

    loss_Model3_Fidelity, weight_Model3_Fidelity = circuit_training(X_train, Y_train, 'Model3_Fidelity')

    Loss_histories_Model3_Fidelity.append(loss_Model3_Fidelity)
    weights_Model3_Fidelity.append(weight_Model3_Fidelity)

f = open('/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Noisy/Loss_histories_and_weights_Noisy_Model3_Fidelity.txt', 'w')
for i in range(5):
    f.write(f'Loss History Noisy Model3 Fidelity {i + 1}:')
    f.write('\n')
    f.write(str(Loss_histories_Model3_Fidelity[i]))
    f.write('\n')
for i in range(5):
    f.write(f'Weights Noisy Model3 Fidelity {i + 1}:')
    f.write('\n')
    f.write(str(weights_Model3_Fidelity[i]))
f.close()
"""

# Part 3: Noisy Model Amplitude

Load dataset for Model Amplitude

In [ ]:
feature_reduction = 'PCA16'
X_train, X_test, Y_train, Y_test = data.data_load_and_process('mnist', feature_reduction=feature_reduction, classes=classes)

In [ ]:
X_train, X_test = torch.tensor(X_train).to(torch.float32), torch.tensor(X_test).to(torch.float32)
Y_train = [-1 if y == 0 else 1 for y in Y_train]
Y_test = [-1 if y == 0 else 1 for y in Y_test]

Train Noisy QCNN Model Amplitude

In [ ]:
"""
for i in range(5):

    f = open('/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Noisy/Loss_histories_and_weights_Noisy_Model_amp_not_trained.txt', 'a')

    loss_Model_amp_not_trained, weight_Model_amp_not_trained = circuit_training(X_train, Y_train, 'Model_amp_not_trained')

    f.write(f'Loss History Noisy Model Amplitude Not Trained{i + 1}:')
    f.write('\n')
    f.write(str(loss_Model_amp_not_trained))
    f.write('\n')
    f.write(f'Weights Noisy Model Amplitude Not Trained{i + 1}:')
    f.write('\n')
    f.write(str(weight_Model_amp_not_trained))
    f.write('\n')

    f.close()



    f = open('/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Noisy/Loss_histories_and_weights_Noisy_Model_amp.txt', 'a')

    loss_Model_amp, weight_Model_amp = circuit_training(X_train, Y_train, 'Model_amp')

    f.write(f'Loss History Noisy Model Amplitude{i + 1}:')
    f.write('\n')
    f.write(str(loss_Model_amp))
    f.write('\n')
    f.write(f'Weights Noisy Model Amplitude{i + 1}:')
    f.write('\n')
    f.write(str(weight_Model_amp))
    f.write('\n')

    f.close()
"""



# Part 4: Result 

Load the results

In [ ]:
PATH_LH0 = '/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Noisy/Loss_histories_not_trained.txt'
PATH_LH1 = '/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Noisy/Loss_histories_Model1.txt'
PATH_LH2 = '/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Noisy/Loss_histories_Model2.txt'
PATH_LH3 = '/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Noisy/Loss_histories_Model3.txt'
PATH_LH_amp0 = '/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Noisy/Loss_histories_Model_amp_not_trained.txt'
PATH_LH_amp1 = '/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Noisy/Loss_histories_Model_amp.txt'

PATH_w0 = '/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Noisy/weights_not_trained.txt'
PATH_w1 = '/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Noisy/weights_Model1.txt'
PATH_w2 = '/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Noisy/weights_Model2.txt'
PATH_w3 = '/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Noisy/weights_Model3.txt'
PATH_w_amp0 = '/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Noisy/weights_Model_amp_not_trained.txt'
PATH_w_amp1 = '/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Noisy/weights_Model_amp.txt'

def data_read(PATH):
  textfile = open(PATH, 'r')
  data = np.array([])
  for line in textfile:
    a = np.array([float(v) for v in line.strip().split(" ")])
    data = np.append(data, a)
  return data

Loss_histories_not_trained = data_read(PATH_LH0).reshape(5,1000)
Loss_histories_Model1_Fidelity = data_read(PATH_LH1).reshape(5,1000)
Loss_histories_Model2_Fidelity = data_read(PATH_LH2).reshape(5,1000)
Loss_histories_Model3_Fidelity = data_read(PATH_LH3).reshape(5,1000)
Loss_histories_Model_amp_not_trained = data_read(PATH_LH_amp0).reshape(5,1000)
Loss_histories_Model_amp = data_read(PATH_LH_amp1).reshape(5,1000)

weights_not_trained = data_read(PATH_w0).reshape(5,4)
weights_Model1_Fidelity = data_read(PATH_w1).reshape(5,4)
weights_Model2_Fidelity = data_read(PATH_w2).reshape(5,4)
weights_Model3_Fidelity = data_read(PATH_w3).reshape(5,4)
weights_Model_amp_not_trained = data_read(PATH_w_amp0).reshape(5,4)
weights_Model_amp = data_read(PATH_w_amp1).reshape(5,4)

Not_trained_mean, Not_trained_std = Loss_histories_not_trained.mean(axis=0), Loss_histories_not_trained.std(axis=0)
Model1_Fidelity_mean, Model1_Fidelity_std = Loss_histories_Model1_Fidelity.mean(axis=0), Loss_histories_Model1_Fidelity.std(axis=0)
Model2_Fidelity_mean, Model2_Fidelity_std = Loss_histories_Model2_Fidelity.mean(axis=0), Loss_histories_Model2_Fidelity.std(axis=0)
Model3_Fidelity_mean, Model3_Fidelity_std = Loss_histories_Model3_Fidelity.mean(axis=0), Loss_histories_Model3_Fidelity.std(axis=0)
Model_amp_not_trained_mean, Model_amp_not_trained_std = Loss_histories_Model_amp_not_trained.mean(axis=0), Loss_histories_Model_amp_not_trained.std(axis=0)
Model_amp_mean, Model_amp_std = Loss_histories_Model_amp.mean(axis=0), Loss_histories_Model_amp.std(axis=0)

Plot the results for Model 1,2,3

In [ ]:
import seaborn as sns

plt.rcParams['figure.figsize'] = [10, 5]
fig, ax = plt.subplots()
clrs = sns.color_palette("husl", 4)
with sns.axes_style("darkgrid"):
    ax.plot(range(len(Not_trained_mean)), Not_trained_mean, label="No Pre-training", c=clrs[0])
    ax.fill_between(range(len(Not_trained_mean)), Not_trained_mean-Not_trained_std, Not_trained_mean+Not_trained_std, alpha=0.3,facecolor=clrs[0])

    ax.plot(range(len(Model1_Fidelity_mean)), Model1_Fidelity_mean, label="Noisy Model1 Pre-training", c=clrs[1])
    ax.fill_between(range(len(Model1_Fidelity_mean)), Model1_Fidelity_mean-Model1_Fidelity_std, Model1_Fidelity_mean+Model1_Fidelity_std, alpha=0.3,facecolor=clrs[1])
    
    ax.plot(range(len(Model2_Fidelity_mean)), Model2_Fidelity_mean, label="Noisy Model2 Pre-training", c=clrs[2])
    ax.fill_between(range(len(Model2_Fidelity_mean)), Model2_Fidelity_mean-Model2_Fidelity_std, Model2_Fidelity_mean+Model2_Fidelity_std, alpha=0.3,facecolor=clrs[2])

    ax.plot(range(len(Model3_Fidelity_mean)), Model3_Fidelity_mean, label="Noisy Model2 Pre-training", c=clrs[3])
    ax.fill_between(range(len(Model3_Fidelity_mean)), Model3_Fidelity_mean-Model3_Fidelity_std, Model3_Fidelity_mean+Model3_Fidelity_std, alpha=0.3,facecolor=clrs[3])


    ax.plot(range(1000), np.ones(1000) * LB_before_traindata, linestyle='dashed', linewidth=1.5, label="Lower Bound without Pre-training", c=clrs[0])
    ax.plot(range(1000), np.ones(1000) * LB_Model1_traindata, linestyle='dashed', linewidth=1.5, label="Lower Bound with Noisy Model1", c=clrs[1])
    ax.plot(range(1000), np.ones(1000) * LB_Model2_traindata, linestyle='dashed', linewidth=1.5, label="Lower Bound with Noisy Model2", c=clrs[2])
    ax.plot(range(1000), np.ones(1000) * LB_Model3_traindata, linestyle='dashed', linewidth=1.5, label="Lower Bound with Noisy Model3", c=clrs[3])


ax.set_xlabel("Iteration")
ax.set_ylabel("Loss")
ax.set_title("Noisy QCNN Loss History")
ax.legend()

Plot the results for Model Amplitude

In [ ]:
LB_amp_before_traindata = 0.2314
LB_amp_after_traindata = 0.0266
import seaborn as sns

plt.rcParams['figure.figsize'] = [10, 5]
fig, ax = plt.subplots()
clrs = sns.color_palette("husl", 2)
with sns.axes_style("darkgrid"):
    ax.plot(range(len(Model_amp_not_trained_mean)), Model_amp_not_trained_mean, label="No Pre-training", c=clrs[0])
    ax.fill_between(range(len(Model_amp_not_trained_mean)), Model_amp_not_trained_mean-Model_amp_not_trained_std, Model_amp_not_trained_mean+Model_amp_not_trained_std, alpha=0.3,facecolor=clrs[0])

    ax.plot(range(len(Model_amp_mean)), Model_amp_mean, label="With Pre-training", c=clrs[1])
    ax.fill_between(range(len(Model_amp_mean)), Model_amp_mean-Model_amp_std, Model_amp_mean+Model_amp_std, alpha=0.3,facecolor=clrs[1])


    ax.plot(range(1000), np.ones(1000) * LB_amp_before_traindata, linestyle='dashed', linewidth=1.5, label="Lower Bound without Pre-training", c=clrs[0])
    ax.plot(range(1000), np.ones(1000) * LB_amp_after_traindata, linestyle='dashed', linewidth=1.5, label="Lower Bound with Pre-training", c=clrs[1])


ax.set_xlabel("Iteration")
ax.set_ylabel("Loss")
ax.set_title("Noisy QCNN Loss History with Amplitude Embedding")
ax.legend()

Calculate the classification accuracies for Model1,2

In [ ]:
feature_reduction = 'PCA4'
X_train, X_test, Y_train, Y_test = data.data_load_and_process('mnist', feature_reduction=feature_reduction, classes=classes)

X_train, X_test = torch.tensor(X_train).to(torch.float32), torch.tensor(X_test).to(torch.float32)
Y_train = [-1 if y == 0 else 1 for y in Y_train]
Y_test = [-1 if y == 0 else 1 for y in Y_test]

In [ ]:
def accuracy_test(predictions, labels):
    acc = 0
    for l, p in zip(labels, predictions):
        if np.abs(l - p) < 1:
            acc = acc + 1
    return acc / len(labels)


accuracies_not_trained, accuracies_Model1_Fidelity, accuracies_Model2_Fidelity = [], [], []

for i in range(5):
    prediction_not_trained = [Noisy_QCNN_classifier(weights_not_trained[i], x, Trained=False) for x in X_test]
    prediction_Model1_Fidelity = [Noisy_QCNN_classifier(weights_Model1_Fidelity[i], x, Trained='Model1_Fidelity') for x in X_test]
    prediction_Model2_Fidelity = [Noisy_QCNN_classifier(weights_Model2_Fidelity[i], x, Trained='Model2_Fidelity') for x in X_test]
    
    accuracy_not_trained = accuracy_test(prediction_not_trained, Y_test)
    accuracy_Model1_Fidelity = accuracy_test(prediction_Model1_Fidelity, Y_test)
    accuracy_Model2_Fidelity = accuracy_test(prediction_Model2_Fidelity, Y_test)

    accuracies_not_trained.append(accuracy_not_trained)
    accuracies_Model1_Fidelity.append(accuracy_Model1_Fidelity)
    accuracies_Model2_Fidelity.append(accuracy_Model2_Fidelity)

accuracies_not_trained = np.array(accuracies_not_trained)
accuracies_Model1_Fidelity, accuracies_Model2_Fidelity = np.array(accuracies_Model1_Fidelity), np.array(accuracies_Model2_Fidelity)

Calculate the accuracies for Model 3

In [ ]:
feature_reduction = False
X_train, X_test, Y_train, Y_test = data.data_load_and_process('mnist', feature_reduction=feature_reduction, classes=classes)

X_train, X_test = torch.tensor(X_train).to(torch.float32), torch.tensor(X_test).to(torch.float32)
X_train, X_test = X_train.permute(0, 3, 1, 2), X_test.permute(0, 3, 1, 2)
Y_train = [-1 if y == 0 else 1 for y in Y_train]
Y_test = [-1 if y == 0 else 1 for y in Y_test]

In [ ]:
accuracies_Model3_Fidelity = []

for i in range(5):
    prediction_Model3_Fidelity = [Noisy_QCNN_classifier(weights_Model3_Fidelity[i], x, Trained='Model3_Fidelity') for x in X_test]
    accuracy_Model3_Fidelity = accuracy_test(prediction_Model3_Fidelity, Y_test)
    accuracies_Model3_Fidelity.append(accuracy_Model3_Fidelity)

accuracies_Model3_Fidelity =np.array(accuracies_Model3_Fidelity)

Calculate the accuracies for Model Amplitude

In [ ]:
feature_reduction = 'PCA16'
X_train, X_test, Y_train, Y_test = data.data_load_and_process('mnist', feature_reduction=feature_reduction, classes=classes)

X_train, X_test = torch.tensor(X_train).to(torch.float32), torch.tensor(X_test).to(torch.float32)
Y_train = [-1 if y == 0 else 1 for y in Y_train]
Y_test = [-1 if y == 0 else 1 for y in Y_test]

In [ ]:
accuracies_amp_not_trained, accuracies_amp = [], []

for i in range(5):
    prediction_amp_not_trained = [Noisy_QCNN_classifier(weights_Model_amp_not_trained[i], x, Trained='Model_amp_not_trained') for x in X_test]
    prediction_amp = [Noisy_QCNN_classifier(weights_Model_amp[i], x, Trained='Model_amp') for x in X_test]
    

    accuracy_amp_not_trained = accuracy_test(prediction_amp_not_trained, Y_test)
    accuracy_amp = accuracy_test(prediction_amp, Y_test)

    accuracies_amp_not_trained.append(accuracy_amp_not_trained)
    accuracies_amp.append(accuracy_amp)


accuracies_amp_not_trained, accuracies_amp = np.array(accuracies_amp_not_trained), np.array(accuracies_amp)

In [ ]:

print("Noisy QCNN classification accuracies:\n")
print("-------------------------------------\n")
print(f" Accuracy without pre-training: {accuracies_not_trained.mean()} ± {accuracies_not_trained.std()}")
print(f" Accuracy after pre-training with Noisy_Model1_Fidelity: {accuracies_Model1_Fidelity.mean()} ± {accuracies_Model1_Fidelity.std()}")
print(f" Accuracy after pre-training with Noisy_Model2_Fidelity: {accuracies_Model2_Fidelity.mean()} ± {accuracies_Model2_Fidelity.std()}")
print(f" Accuracy after pre-training with Noisy_Model3_Fidelity: {accuracies_Model3_Fidelity.mean()} ± {accuracies_Model3_Fidelity.std()}")
print("\n")
print("\n")
print("Noisy QCNN classification accuracies (Amplitude Embedding):\n")
print("-------------------------------------\n")
print(f" Accuracy (Amplitude) without pre-training: {accuracies_amp_not_trained.mean()} ± {accuracies_amp_not_trained.std()}")
print(f" Accuracy (Amplitude) with pre-training: {accuracies_amp.mean()} ± {accuracies_amp.std()}")

